# 使用 openai-agent 的 python-sdk 來 透過 Google Gemini Model API 建立各種 Agents

> 作者: Simon Liu

## I. 安裝 OpenAI Agents python package

In [ ]:
!pip install -q openai-agents

## II. 設定 Agent SDK ，並且將 Google Gemini API Key 等資訊設定進去。

In [3]:
import os
import asyncio

from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled

In [5]:
from google.colab import userdata

BASE_URL = os.getenv("EXAMPLE_BASE_URL") or "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("EXAMPLE_API_KEY") or userdata.get('GOOGLE_API_KEY')
MODEL_NAME = os.getenv("EXAMPLE_MODEL_NAME") or "gemini-2.0-flash"

if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError(
        "Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code."
    )

In [6]:
client = AsyncOpenAI(base_url=BASE_URL, api_key=API_KEY)
set_tracing_disabled(disabled=True)

## III. 建立第一個 Agent: 取得地區天氣（非真實串接，只是吐回文字）

In [11]:
@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."


async def main(prompt = "What's the weather in Tokyo?"):
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="請使用繁體中文回覆結果",
        model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
        tools=[get_weather],
    )

    result = await Runner.run(agent, prompt)
    print(result.final_output)

In [18]:
await main("請問台北的天氣如何？")

[debug] getting weather for 台北
台北的天氣是晴朗的。



## IV. 中階 Agnet: 四則運算 Agent

In [19]:
@function_tool
def calculate(a: float, b: float, operator: str):
    print(f"[debug] calculating: {a} {operator} {b}")
    if operator == '+':
        return f"結果是 {a + b}"
    elif operator == '-':
        return f"結果是 {a - b}"
    elif operator == '*':
        return f"結果是 {a * b}"
    elif operator == '/':
        if b == 0:
            return "錯誤：除數不能為零。"
        return f"結果是 {a / b}"
    else:
        return "錯誤：不支援的運算符，請使用 '+', '-', '*', '/' 其中之一。"

async def main(prompt="幫我算 12 除以 4"):
    agent = Agent(
        name="Calculator",
        instructions="請使用繁體中文回覆結果",
        model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
        tools=[calculate],
    )

    result = await Runner.run(agent, prompt)
    print(result.final_output)


In [21]:
await main("請問312除以四的平方，然後再加555是多少？")

[debug] calculating: 4.0 * 4.0
[debug] calculating: 312.0 / 16.0
[debug] calculating: 19.5 + 555.0
312 除以四的平方，然後再加 555 是 574.5。



## V. 高階 Agents: 透過 xgboost 進行模型訓練

In [30]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib

@function_tool
def train_xgboost_model(
    data_path: str,
    target_column: str,
    model_output_path: str = "xgboost_model.pkl"
):
    """
    訓練一個 XGBoost 分類模型。

    Parameters:
    - data_path: 資料的 CSV 檔案路徑
    - target_column: 目標變數欄位名稱
    - model_output_path: 訓練好的模型儲存檔案路徑

    Returns:
    - 模型訓練的準確率與儲存位置
    """
    print(f"[debug] loading data from {data_path}")
    df = pd.read_csv(data_path)

    if target_column not in df.columns:
        return f"錯誤：指定的目標欄位 '{target_column}' 不存在於資料集中。"

    X = df.drop(columns=[target_column])
    y = df[target_column]

    # 將非數值類別轉為整數編碼
    if y.dtype == 'object' or y.dtype.name == 'category':
        print("[debug] encoding target labels")
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)
        # 可選：儲存 label encoder，供預測階段使用
        joblib.dump(label_encoder, model_output_path.replace(".pkl", "_label_encoder.pkl"))

    print(f"[debug] splitting data")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"[debug] training XGBoost model")
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    print(f"[debug] evaluating model")
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print(f"[debug] saving model to {model_output_path}")
    joblib.dump(model, model_output_path)

    return f"模型訓練完成。準確率為 {acc:.2%}，模型與 LabelEncoder 分別儲存於 {model_output_path} 與 {model_output_path.replace('.pkl', '_label_encoder.pkl')}"


In [41]:
async def main(prompt):
    agent = Agent(
        name="MLTrainer",
        instructions="請使用繁體中文回覆結果",
        model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
        tools=[train_xgboost_model],
    )

    result = await Runner.run(agent, prompt)
    print(result.final_output)


### Case 1: Iris dataset 訓練

In [38]:
# 創建相關資料夾

!mkdir data/ model/

In [40]:
# 下載 iris dataset csv 檔案

!curl --output ./data/iris.csv \
    --url https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3858  100  3858    0     0  27656      0 --:--:-- --:--:-- --:--:-- 27755


In [42]:
# 下 Prompt 來進行模型訓練

await main("請用檔案路徑 ./data/iris.csv 訓練一個目標欄位為 species 的 XGBoost 模型，模型訓練好請存到 ./model/iris_model.pkl")

[debug] loading data from ./data/iris.csv
[debug] encoding target labels
[debug] splitting data
[debug] training XGBoost model


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:46:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[debug] evaluating model
[debug] saving model to ./model/iris_model.pkl
模型訓練完成。準確率為 100.00%，模型與 LabelEncoder 分別儲存於 ./model/iris_model.pkl 與 ./model/iris_model_label_encoder.pkl。



### Case 2: Wine dataset 分類演算法模型訓練

In [43]:
!curl --output ./data/wine.csv \
    --url https://gist.githubusercontent.com/tijptjik/9408623/raw/b237fa5848349a14a14e5d4107dc7897c21951f5/wine.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10889  100 10889    0     0  40949      0 --:--:-- --:--:-- --:--:-- 40936


In [45]:
await main("請用檔案路徑 ./data/wine.csv 訓練一個目標欄位為 Wine 的 XGBoost 模型，模型訓練好請存到 ./model/wine_model.pkl")

[debug] loading data from ./data/wine.csv
[debug] splitting data
[debug] training XGBoost model
模型訓練發生錯誤。錯誤訊息指出從目標變數 `Wine` 推斷出的類別無效。預期的類別是 `[0 1 2]`，但實際得到的類別是 `[1 2 3]`。這表示目標變數的類別編碼從 1 開始，而不是從 0 開始。XGBoost 模型預期類別從 0 開始編碼。

為了修正這個問題，您需要預先處理您的資料，將 `Wine` 欄位中的類別編碼從 `[1 2 3]` 轉換為 `[0 1 2]`。您可以使用 Python 和 Pandas 讀取 CSV 檔案，然後將 `Wine` 欄位的值減 1。

如果修正資料後，您仍然遇到問題，請檢查資料路徑、目標欄位名稱和模型輸出路徑是否正確。



-> 你應該會得到類別無效的錯誤訊息，這是正常的，因為 target label 跟 iris dataset 是不一樣的，你有兩個解法：

1. 自己整理成一樣的資料樣子
2. 寫一個 function tool 來進行資料整理

因為要示範 Agent ，所以這邊示範第二種。

In [46]:
@function_tool
def normalize_class_labels(
    data_path: str,
    target_column: str,
    output_path: str = None
):
    """
    將分類欄位中的類別值調整為從 0 開始的整數（例如從 [1,2,3] → [0,1,2]）。

    Parameters:
    - data_path: 原始 CSV 檔案路徑
    - target_column: 需要標準化的目標欄位名稱
    - output_path: 轉換後資料儲存路徑（如未提供，將覆寫原始檔案）

    Returns:
    - 新類別對應表與儲存檔案路徑
    """
    print(f"[debug] loading data from {data_path}")
    df = pd.read_csv(data_path)

    if target_column not in df.columns:
        return f"錯誤：目標欄位 '{target_column}' 不存在於資料中。"

    print(f"[debug] normalizing target column '{target_column}'")
    unique_classes = sorted(df[target_column].unique())
    class_mapping = {label: idx for idx, label in enumerate(unique_classes)}
    df[target_column] = df[target_column].map(class_mapping)

    output_file = output_path if output_path else data_path
    df.to_csv(output_file, index=False)

    return f"類別欄位已標準化為從 0 開始的整數。對應關係為：{class_mapping}。資料已儲存至 {output_file}"


In [47]:
async def main(prompt):
    agent = Agent(
        name="MLTrainer",
        instructions="請使用繁體中文回覆結果",
        model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
        tools=[train_xgboost_model, normalize_class_labels],
    )

    result = await Runner.run(agent, prompt)
    print(result.final_output)


In [48]:
await main("請用檔案路徑 ./data/wine.csv 訓練一個目標欄位為 Wine 的 XGBoost 模型，模型訓練好請存到 ./model/wine_model.pkl")

[debug] loading data from ./data/wine.csv
[debug] splitting data
[debug] training XGBoost model
[debug] loading data from ./data/wine.csv
[debug] normalizing target column 'Wine'
[debug] loading data from ./data/wine_normalized.csv
[debug] splitting data
[debug] training XGBoost model
[debug] evaluating model
[debug] saving model to ./model/wine_model.pkl


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:51:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost 模型已成功訓練！準確率為 94.44%，模型儲存於 `./model/wine_model.pkl`。



你應該就有得到正確訓練模型的訊息了。